In [ ]:
pip install python-binance

In [26]:
from binance.client import Client
import pandas as pd

# For public price data, you can leave keys empty
client = Client(api_key='', api_secret='')

# Example: Get 100 daily candles for BTC/USDT
klines = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1DAY, limit=100)

# Convert to DataFrame
df = pd.DataFrame(klines, columns=[
    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close time', 'Quote asset volume', 'Number of trades',
    'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
])
df['Date'] = pd.to_datetime(df['Open time'], unit='ms')
df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]


In [ ]:
print(df.head())

In [ ]:
pip install pandas-ta

In [ ]:
pip install ta

In [ ]:
import ta

df = df.dropna()
df['RSI_14'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()
df['SMA_10'] = ta.trend.SMAIndicator(close=df['Close'], window=10).sma_indicator()
df['EMA_10'] = ta.trend.EMAIndicator(close=df['Close'], window=10).ema_indicator()

# Bollinger Bands
bb = ta.volatility.BollingerBands(close=df['Close'])
df['BB_High'] = bb.bollinger_hband()
df['BB_Low'] = bb.bollinger_lband()

# MACD
macd = ta.trend.MACD(close=df['Close'])
df['MACD'] = macd.macd()

# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'])
df['Stoch_K'] = stoch.stoch()
df['Stoch_D'] = stoch.stoch_signal()

# CCI (Commodity Channel Index)
cci = ta.trend.CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'])
df['CCI'] = cci.cci()

# ADX (Average Directional Index)
adx = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'])
df['ADX'] = adx.adx()



In [ ]:
print(df.tail(10))  # shows last 10 rows

In [ ]:
pip install newsapi-python vaderSentiment

In [ ]:
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd


newsapi = NewsApiClient(api_key='fe27f943772a4235a29f751edabae735')

all_articles = newsapi.get_everything(
    q='Bitcoin',
    language='en',
    sort_by='publishedAt',
    page_size=100
)
news_df = pd.DataFrame(all_articles['articles'])
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()
news_df['sentiment_score'] = news_df['title'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Add 'Date' column for grouping
news_df['Date'] = news_df['publishedAt'].dt.date
news_df['Date'] = pd.to_datetime(news_df['Date'])

# Group by date to get average sentiment per day
daily_sentiment = news_df.groupby('Date')['sentiment_score'].mean().reset_index()
daily_sentiment.rename(columns={'sentiment_score': 'Sentiment'}, inplace=True)


In [ ]:
news_df.head()


In [ ]:
# Convert dates to same format
daily_sentiment['Date'] = pd.to_datetime(daily_sentiment['Date'])

# Merge on 'Date'
df = pd.merge(df, daily_sentiment, on='Date', how='left')

#fill for na
df['Sentiment'] = df['Sentiment'].fillna(0)


In [ ]:
print(df[['Date', 'Close', 'Sentiment']].tail(10))  # last 10 rows
print(df.describe())  # numerical summary (mean, std, min, etc.)

In [37]:
#cretae a target variable
# Create binary target: 1 if tomorrow's close is higher than today's, else 0
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)


In [38]:
features = [
    'RSI_14', 'SMA_10', 'EMA_10', 'Sentiment',
    'BB_High', 'BB_Low', 'MACD', 'Stoch_K', 'Stoch_D', 'CCI', 'ADX'
]

df = df.dropna()  # drop rows with any NaN values

X = df[features]
y = df['Target']

In [ ]:
%pip install scikit-learn

In [ ]:
#Train/test Split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)) 
print(classification_report(y_test, y_pred))
